<a href="https://www.kaggle.com/code/rakibulhasanshaon69/freminghum-last-and-final?scriptVersionId=207794123" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from sklearn.feature_selection import mutual_info_regression
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.exceptions import NotFittedError

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, BaggingClassifier, HistGradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids, NearMiss, EditedNearestNeighbours
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline

In [2]:
data= pd.read_csv('/kaggle/input/framingham/framingham.csv')
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,male,39,4.0,No,0.0,0.0,0,0,No,195.0,106.0,70.0,26.97,80.0,77.0,0
1,female,46,2.0,No,0.0,0.0,0,0,No,250.0,121.0,81.0,28.73,95.0,76.0,0
2,male,48,1.0,Yes,20.0,0.0,0,0,No,245.0,127.5,80.0,25.34,75.0,70.0,0
3,female,61,3.0,Yes,30.0,0.0,0,1,No,225.0,150.0,95.0,28.58,65.0,103.0,1
4,female,46,3.0,Yes,23.0,0.0,0,0,No,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,female,48,2.0,Yes,20.0,NaN,0,0,No,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,female,44,1.0,Yes,15.0,0.0,0,0,No,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,female,52,2.0,No,0.0,0.0,0,0,No,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,male,40,3.0,No,0.0,0.0,0,1,No,185.0,141.0,98.0,25.60,67.0,72.0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sex              4240 non-null   object 
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   object 
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   object 
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(4), object(3)
memory usage: 530.1+ KB


In [4]:
data.isnull().sum()

Sex                  0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [5]:
unique_values= {}
for i in data.columns:
    unique_values[i]= data[i].nunique()
    
pd.DataFrame(unique_values,index= ['unique_value']).T

,unique_value
Sex,2
age,39
education,4
currentSmoker,2
cigsPerDay,33
BPMeds,2
prevalentStroke,2
prevalentHyp,2
diabetes,2
totChol,248


In [6]:
round(data.describe().T)

,count,mean,std,min,25%,50%,75%,max
age,4240.0,50.0,9.0,32.0,42.0,49.0,56.0,70.0
education,4135.0,2.0,1.0,1.0,1.0,2.0,3.0,4.0
cigsPerDay,4211.0,9.0,12.0,0.0,0.0,0.0,20.0,70.0
BPMeds,4187.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
prevalentStroke,4240.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
prevalentHyp,4240.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
totChol,4190.0,237.0,45.0,107.0,206.0,234.0,263.0,696.0
sysBP,4240.0,132.0,22.0,84.0,117.0,128.0,144.0,295.0
diaBP,4240.0,83.0,12.0,48.0,75.0,82.0,90.0,142.0
BMI,4221.0,26.0,4.0,16.0,23.0,25.0,28.0,57.0


In [7]:
data.columns

Index(['Sex', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

In [8]:
data['Sex'] = data['Sex'].map({'male': 1, 'female': 0})
data['currentSmoker']= data['currentSmoker'].map({'Yes':1, 'No':0})
data['diabetes']= data['diabetes'].map({'Yes':1, 'No':0})

In [9]:
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [10]:
data['cigsPerDay'] = data['cigsPerDay'].fillna(data['cigsPerDay'].mean())
data['BPMeds'] = data['BPMeds'].fillna(data['BPMeds'].median())
data['totChol'] = data['totChol'].fillna(data['totChol'].mean())
data['BMI'] = data['BMI'].fillna(data['BMI'].median())
data['heartRate'] = data['heartRate'].fillna(data['heartRate'].median())
data['glucose'] = data['glucose'].fillna(data['glucose'].median())
data['education'] = data['education'].fillna(data['education'].mode()[0])
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,78.0,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sex              4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4240 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4240 non-null   float64
 5   BPMeds           4240 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   int64  
 9   totChol          4240 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4240 non-null   float64
 13  heartRate        4240 non-null   float64
 14  glucose          4240 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 530.1 KB


# Removing Outlayer

In [12]:
data['totChol'] = data['totChol'].apply(lambda x: 300 if x > 300 else x)
data['cigsPerDay'] = data['cigsPerDay'].apply(lambda x: 30 if x >30 else x)
data['sysBP'] = data['sysBP'].apply(lambda x: 170 if x >170 else x)
data['glucose'] = data['glucose'].apply(lambda x: 200 if x > 200 else x)
# data['heartRate'] = data['heartRate'].apply(lambda x: 10 if x > 100 else x)

In [13]:
round(data.describe().T)

,count,mean,std,min,25%,50%,75%,max
Sex,4240.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
age,4240.0,50.0,9.0,32.0,42.0,49.0,56.0,70.0
education,4240.0,2.0,1.0,1.0,1.0,2.0,3.0,4.0
currentSmoker,4240.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
cigsPerDay,4240.0,8.0,11.0,0.0,0.0,0.0,20.0,30.0
BPMeds,4240.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
prevalentStroke,4240.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
prevalentHyp,4240.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
diabetes,4240.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
totChol,4240.0,234.0,39.0,107.0,206.0,234.0,262.0,300.0


In [14]:
data.isnull().sum()

Sex                0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [15]:
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,78.0,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [16]:
negative_class = data[data['TenYearCHD'] == 0]
positive_class = data[data['TenYearCHD'] == 1]

negative_sample = negative_class.sample(n=60, random_state=42)
positive_sample = positive_class.sample(n=40, random_state=42)  

df = pd.concat([negative_sample, positive_sample])
data = data.drop(df.index)  
df.head(100)

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
3615,1,38,3.0,1,20.0,0.0,0,0,0,270.0,120.0,75.0,23.76,83.0,96.0,0
1488,1,37,4.0,1,30.0,0.0,0,0,0,254.0,122.5,82.5,23.87,88.0,83.0,0
3824,0,39,3.0,1,8.0,0.0,0,0,0,192.0,109.0,61.0,23.36,64.0,84.0,0
1267,0,46,3.0,0,0.0,0.0,0,0,0,190.0,128.0,74.0,23.01,95.0,78.0,0
2022,0,45,2.0,1,9.0,0.0,0,0,0,208.0,137.0,82.0,24.35,85.0,78.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,0,55,2.0,1,20.0,0.0,0,0,0,250.0,138.0,87.0,25.33,95.0,78.0,1
619,0,51,1.0,1,20.0,0.0,0,0,0,264.0,139.5,89.0,29.38,70.0,76.0,1
2404,1,51,1.0,1,15.0,0.0,0,0,0,220.0,125.0,82.0,24.10,60.0,73.0,1
1298,1,40,1.0,0,0.0,0.0,0,1,0,175.0,170.0,59.0,27.99,70.0,75.0,1


In [17]:
data

,Sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,0.0,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,78.0,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [18]:
import numpy as np
import pandas as pd
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

target = data['TenYearCHD']
features = data.drop(['TenYearCHD'], axis=1)
X = features.values  
y = target.values 

extra_trees_model = ExtraTreesClassifier(random_state=63, class_weight={0: 1, 1: 6.9})
under_sampling_methods = {
    "RandomUnderSampler": RandomUnderSampler(),
    "TomekLinks": TomekLinks(),
    "ClusterCentroids": ClusterCentroids(),
    "NearMiss": NearMiss(),
    "EditedNearestNeighbours": EditedNearestNeighbours()
}
over_sampling_methods = {
    "ADASYN": ADASYN(),
    "SMOTE": SMOTE(),
    "RandomOverSampler": RandomOverSampler()
}
results = []
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
for over_name, over_sampler in over_sampling_methods.items():
    for under_name, under_sampler in under_sampling_methods.items():
        pipeline = Pipeline(steps=[('over', over_sampler), ('under', under_sampler)])
        X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)
        
        model = ExtraTreesClassifier(random_state=63, class_weight={0: 1, 1: 6.9})
        model.fit(X_resampled, y_resampled)
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)
        f1 = report['weighted avg']['f1-score']
        precision = report['weighted avg']['precision']
        recall = report['weighted avg']['recall']
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (None, None, None, None)
        specificity = tn / (tn + fp) if tn is not None else None

        results.append({
            "Over-Sampling Method": over_name,
            "Under-Sampling Method": under_name,
            "Accuracy": accuracy,
            "F1 Score": f1,
            "Precision": precision,
            "Recall": recall,
            "Specificity": specificity,
            "Confusion Matrix": cm
        })

results_df = pd.DataFrame(results)
print(results_df)
results_df.to_csv("combined_sampling_results.csv", index=False)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/imblearn/under_sampling/_prototype_generation/_cluster_centroids.py:178: ConvergenceWarning: Number of distinct clusters (477) found smaller than n_clusters (2835). Possibly due to duplicate points in X.
  self.estimator_.fit(_safe_indexing(X, target_class_indi

   Over-Sampling Method    Under-Sampling Method  Accuracy  F1 Score  \
0                ADASYN       RandomUnderSampler  0.798309  0.768095   
1                ADASYN               TomekLinks  0.806763  0.780847   
2                ADASYN         ClusterCentroids  0.811594  0.783975   
3                ADASYN                 NearMiss  0.806763  0.780847   
4                ADASYN  EditedNearestNeighbours  0.650966  0.697106   
5                 SMOTE       RandomUnderSampler  0.807971  0.777949   
6                 SMOTE               TomekLinks  0.804348  0.773109   
7                 SMOTE         ClusterCentroids  0.811594  0.783975   
8                 SMOTE                 NearMiss  0.806763  0.779655   
9                 SMOTE  EditedNearestNeighbours  0.812802  0.782282   
10    RandomOverSampler       RandomUnderSampler  0.842995  0.784975   
11    RandomOverSampler               TomekLinks  0.849034  0.795980   
12    RandomOverSampler         ClusterCentroids  0.839372  0.77

In [19]:
# if __name__ == "__main__":   
#     target = data['TenYearCHD']
#     columns_to_drop = ['TenYearCHD']
#     features = data.drop(columns_to_drop, axis=1)

#     X = features.values
#     y = target.values
    
#     count_zeros_original = np.sum(y == 0)
#     count_ones_original = np.sum(y == 1)

#     majority_count = count_zeros_original  
#     minority_count = count_ones_original   

#     desired_minority_count = majority_count
#     num_samples_to_oversample = desired_minority_count - minority_count - 100

#     over = ADASYN(sampling_strategy={1: desired_minority_count}, random_state=63)  
#     under = RandomUnderSampler(sampling_strategy={0: majority_count}, random_state=63)  

#     pipeline = Pipeline(steps=[('over', over), ('under', under)])

#     X_resampled, y_resampled = pipeline.fit_resample(X, y)
#     x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, stratify=y_resampled, random_state=42)

#     model = RandomForestClassifier(random_state=42)
#     model.fit(x_train, y_train)
#     y_pred_proba = model.predict_proba(x_test)[:, 1]
#     threshold = 0.5
#     y_pred_adjusted = (y_pred_proba >= threshold).astype(int)
#     print(classification_report(y_test, y_pred_adjusted))

# #               precision    recall  f1-score   support

# #            0       0.94      0.96      0.95       320
# #            1       0.96      0.94      0.95       328

# #     accuracy                           0.95       648
# #    macro avg       0.95      0.95      0.95       648
# # weighted avg       0.95      0.95      0.95       648

In [20]:
if __name__ == "__main__":
    target = data['TenYearCHD']
    features = data.drop(['TenYearCHD'], axis=1)

    X = features.values
    y = target.values
    
    count_zeros_original = np.sum(y == 0)
    majority_count = count_zeros_original  
    desired_minority_count = majority_count
    pipelines = {
        "ADASYN + EditedNearestNeighbours": Pipeline([
            ('over', ADASYN(sampling_strategy={1: desired_minority_count}, random_state=63)),
            ('under', EditedNearestNeighbours())
        ]),
        "RandomOverSampler + EditedNearestNeighbours": Pipeline([
            ('over', RandomOverSampler(sampling_strategy={1: desired_minority_count})),
            ('under', EditedNearestNeighbours())
        ])
    }
    results = {}

    for name, pipeline in pipelines.items():
        X_resampled, y_resampled = pipeline.fit_resample(X, y)
        x_train, x_test, y_train, y_test = train_test_split(
            X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
        )
        model = RandomForestClassifier(random_state=42)
        model.fit(x_train, y_train)
        y_pred_proba = model.predict_proba(x_test)[:, 1]
        y_pred_adjusted = (y_pred_proba >= 0.5).astype(int)
        results[name] = classification_report(y_test, y_pred_adjusted, output_dict=True)
        print(f"\nResults for {name}:")
        print(classification_report(y_test, y_pred_adjusted))


Results for ADASYN + EditedNearestNeighbours:
              precision    recall  f1-score   support

           0       0.92      0.84      0.87       379
           1       0.92      0.96      0.94       705

    accuracy                           0.92      1084
   macro avg       0.92      0.90      0.91      1084
weighted avg       0.92      0.92      0.92      1084


Results for RandomOverSampler + EditedNearestNeighbours:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       707
           1       0.96      1.00      0.98       675

    accuracy                           0.98      1382
   macro avg       0.98      0.98      0.98      1382
weighted avg       0.98      0.98      0.98      1382



In [21]:
from collections import Counter

target = data['TenYearCHD']
features = data.drop(['TenYearCHD'], axis=1)
X = features.values
y = target.values
initial_counts = Counter(y)
print("Original class distribution:", initial_counts)
majority_count = initial_counts[0]
pipeline = Pipeline([
    ('over', RandomOverSampler(sampling_strategy={1: majority_count})),
    ('under', EditedNearestNeighbours())
])
X_resampled, y_resampled = pipeline.fit_resample(X, y)
resampled_counts = Counter(y_resampled)
print("Resampled class distribution:", resampled_counts)
synthetic_samples_created = resampled_counts[1] - initial_counts[1]
print("Number of synthetic samples created for the minority class:", synthetic_samples_created)
x_train, x_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

model = RandomForestClassifier(n_estimators=500, random_state=42)
model.fit(x_train, y_train)
y_pred_proba = model.predict_proba(x_test)[:, 1]
y_pred_adjusted = (y_pred_proba >= 0.5).astype(int)
print("\nClassification Report for RandomOverSampler + EditedNearestNeighbours:")
print(classification_report(y_test, y_pred_adjusted))

Original class distribution: Counter({0: 3536, 1: 604})
Resampled class distribution: Counter({0: 3536, 1: 3360})
Number of synthetic samples created for the minority class: 2756

Classification Report for RandomOverSampler + EditedNearestNeighbours:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98       708
           1       0.95      1.00      0.98       672

    accuracy                           0.98      1380
   macro avg       0.98      0.98      0.98      1380
weighted avg       0.98      0.98      0.98      1380



In [22]:
# from collections import Counter
# target = data['TenYearCHD']
# features = data.drop(['TenYearCHD'], axis=1)
# X = features.values
# y = target.values
# initial_counts = Counter(y)
# desired_minority_count = 2700
# pipeline = Pipeline([
#     ('smote', SMOTE(sampling_strategy={1: desired_minority_count})),
#     ('enn', EditedNearestNeighbours())
# ])
# X_resampled, y_resampled = pipeline.fit_resample(X, y)
# resampled_counts = Counter(y_resampled)
# synthetic_samples_created = resampled_counts[1] - initial_counts[1]
# x_train, x_test, y_train, y_test = train_test_split(
#     X_resampled, y_resampled, test_size=0.1, stratify=y_resampled, random_state=42
# )
# model = RandomForestClassifier(n_estimators=500, random_state=42)
# model.fit(x_train, y_train)
# y_pred_proba = model.predict_proba(x_test)[:, 1]
# y_pred_adjusted = (y_pred_proba >= 0.5).astype(int)
# print(initial_counts)
# print(resampled_counts)
# print(synthetic_samples_created)
# print(classification_report(y_test, y_pred_adjusted))

In [23]:
classifiers = {
    "RandomForest": RandomForestClassifier(n_estimators=500, criterion="entropy", max_features="auto", bootstrap=False, random_state=63, class_weight={0: 3, 1: 21}),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42, class_weight={0: 1, 1: 6.9}),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "HistGradientBoosting": HistGradientBoostingClassifier(max_iter=1600, learning_rate=0.2, max_leaf_nodes=40, class_weight={0: 1, 1: 6.9}, random_state=63),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(random_state=42),
    "GaussianNB": GaussianNB(),
    "BernoulliNB": BernoulliNB(),
    "KNeighbors": KNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(n_estimators=500, learning_rate=0.2, reg_lambda=0.0, random_state=42, class_weight={0: 1, 1: 11}),
    "MLPClassifier": MLPClassifier(random_state=42)
}

def evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test):
    results = []
    for name, clf in classifiers.items():
        try:
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)
            cm = confusion_matrix(y_test, y_pred)
            tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (None, None, None, None)
            specificity = tn / (tn + fp) if tn is not None else None

            metrics = {
                'Model': name,
                'Accuracy': accuracy_score(y_test, y_pred),
                'F1 Score': f1_score(y_test, y_pred, average='weighted'),
                'Precision': classification_report(y_test, y_pred, output_dict=True)['weighted avg']['precision'],
                'Recall': classification_report(y_test, y_pred, output_dict=True)['weighted avg']['recall'],
                'Specificity': specificity,
                'ROC-AUC': roc_auc_score(y_test, clf.predict_proba(x_test)[:, 1]) if hasattr(clf, "predict_proba") else None,
                'Confusion Matrix': cm
            }
            results.append(metrics)
        except Exception as e:
            print(f"Error with classifier {name}: {e}")
    return pd.DataFrame(results)

results_df = evaluate_classifiers(classifiers, x_train, x_test, y_train, y_test)
print(results_df)
results_df.to_csv("ResultsForTest_size=0.1.csv", index=False)

/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[LightGBM] [Info] Number of positive: 2688, number of negative: 2828
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003073 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 5516, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.912705 -> initscore=2.347123
[LightGBM] [Info] Start training from score 2.347123
                   Model  Accuracy  F1 Score  Precision    Recall  \
0           RandomForest  0.985507  0.985510   0.985926  0.985507   
1               AdaBoost  0.665942  0.665999   0.666308  0.665942   
2             ExtraTrees  0.992029  0.992030   0.992157  0.992029   
3       GradientBoosting  0.707246  0.707286   0.707828  0.707246   
4                Bagging  0.955797  0.955773   0.958794  0.955797   
5   HistGradientBoosting  0.963043

In [24]:
model = ExtraTreesClassifier(random_state=42, class_weight={0: 1, 1: 6.9})
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
train_preds = model.predict(x_train)
train_accuracy = accuracy_score(y_train, train_preds)
print("Training Accuracy: {:.2f}%".format(train_accuracy *100))
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(test_accuracy *100))
print(classification_report(y_test, y_pred))

Training Accuracy: 100.00%
Test Accuracy: 99.20%
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       708
           1       0.98      1.00      0.99       672

    accuracy                           0.99      1380
   macro avg       0.99      0.99      0.99      1380
weighted avg       0.99      0.99      0.99      1380

